# MIBI-TOF in SpatialData

This notebook is configured for a local SpatialData archive downloaded from the scverse docs. It extracts the bundled `data.zarr` store into a repo-local cache and then opens it with `spatialdata.read_zarr()`.

Configured archive: `/Users/chrislangseth/Downloads/mibitof_spatialdata_0.7.1.zip`

In [ ]:
from pathlib import Path
import os
import zipfile
from pprint import pprint

import pandas as pd
import spatialdata as sd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc

In [ ]:
ZIP_PATH = Path(os.environ.get("SPATIALDATA_ZIP_PATH", "/Users/chrislangseth/Downloads/mibitof_spatialdata_0.7.1.zip"))
EXTRACT_ROOT = Path(os.environ.get("SPATIALDATA_EXTRACT_ROOT", "/Users/chrislangseth/work/karolinska_institutet/projects/KaroSpaceDataWrangling/data/spatialdata_cache/mibitof_spatialdata_0_7_1"))
ZARR_PATH = EXTRACT_ROOT / "data.zarr"

ZIP_PATH, EXTRACT_ROOT, ZARR_PATH

In [ ]:
if not ZIP_PATH.exists():
    raise FileNotFoundError(f"Archive not found: {ZIP_PATH}")

EXTRACT_ROOT.mkdir(parents=True, exist_ok=True)

if not ZARR_PATH.exists():
    with zipfile.ZipFile(ZIP_PATH) as zf:
        members = [name for name in zf.namelist() if name.startswith("data.zarr/")]
        if not members:
            raise ValueError(f"Archive does not contain data.zarr/: {ZIP_PATH}")
        zf.extractall(EXTRACT_ROOT, members=members)
        print(f"Extracted {len(members)} members to {EXTRACT_ROOT}")
else:
    print(f"Using existing extracted store: {ZARR_PATH}")

sdata = sd.read_zarr(ZARR_PATH)

for table in sdata.tables.values():
    table.var_names_make_unique()

sdata

In [ ]:
summary = {
    "images": list(sdata.images),
    "labels": list(sdata.labels),
    "points": list(sdata.points),
    "shapes": list(sdata.shapes),
    "tables": list(sdata.tables),
}

pprint(summary)

In [ ]:
pd.DataFrame(
    [
        {
            "table": name,
            "n_obs": table.n_obs,
            "n_vars": table.n_vars,
            "obs_columns": ", ".join(map(str, table.obs.columns[:12])),
            "var_columns": ", ".join(map(str, table.var.columns[:12])),
            "obsm_keys": ", ".join(map(str, table.obsm_keys())),
        }
        for name, table in sdata.tables.items()
    ]
)

## Convert To AnnData

The next cell selects a table from the SpatialData object, converts it to an `AnnData` object called `adata`, and ensures `adata.obsm["spatial"]` is present for downstream Scanpy/Squidpy workflows.

Set `SPATIALDATA_TABLE` in the environment if you want to force a specific table name. For the Visium HD archive, the default is `cell_segmentations` when available.


In [ ]:
PREFERRED_TABLE = os.environ.get("SPATIALDATA_TABLE")

if PREFERRED_TABLE is not None:
    if PREFERRED_TABLE not in sdata.tables:
        raise KeyError(
            f"Requested table {PREFERRED_TABLE!r} not found. Available tables: {list(sdata.tables)}"
        )
    TABLE_NAME = PREFERRED_TABLE
elif "cell_segmentations" in sdata.tables:
    TABLE_NAME = "cell_segmentations"
elif "table" in sdata.tables:
    TABLE_NAME = "table"
else:
    TABLE_NAME = next(iter(sdata.tables))

adata = sdata.tables[TABLE_NAME].copy()
adata.obs_names_make_unique()
adata.var_names_make_unique()

if "spatial" not in adata.obsm:
    spatial_candidates = [
        ("x", "y"),
        ("X", "Y"),
        ("center_x", "center_y"),
        ("centroid_x", "centroid_y"),
        ("array_col", "array_row"),
        ("pxl_col_in_fullres", "pxl_row_in_fullres"),
    ]
    for x_key, y_key in spatial_candidates:
        if x_key in adata.obs.columns and y_key in adata.obs.columns:
            adata.obsm["spatial"] = adata.obs[[x_key, y_key]].to_numpy()
            break

if "spatial" not in adata.obsm:
    raise ValueError(
        "Could not determine spatial coordinates for `adata`. "
        "Inspect `adata.obs.columns` and `adata.obsm.keys()` to map coordinates explicitly."
    )

adata.uns["spatialdata_source"] = {
    "zip_path": str(ZIP_PATH),
    "zarr_path": str(ZARR_PATH),
    "table_name": TABLE_NAME,
}

print(f"Selected table: {TABLE_NAME}")
print(adata)
print(f"obsm keys: {list(adata.obsm.keys())}")
adata

## Basic Processing

This section follows the same general pattern as the existing `cosmx.ipynb` workflow, but uses a standard Scanpy pipeline that should work across these SpatialData-derived `AnnData` objects.


In [ ]:
sample_key_candidates = ["sample_id", "library_id", "sample", "fov_labels", "region"]
existing_sample_key = next((key for key in sample_key_candidates if key in adata.obs.columns), None)

if existing_sample_key is None:
    adata.obs["sample_id"] = TABLE_NAME
else:
    adata.obs["sample_id"] = adata.obs[existing_sample_key].astype(str)

adata.obs["sample_id"] = adata.obs["sample_id"].astype("category")
adata

In [ ]:
adata.var["mt"] = adata.var_names.str.upper().str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True)

qc_cols = [col for col in ["sample_id", "total_counts", "n_genes_by_counts", "pct_counts_mt"] if col in adata.obs.columns]
adata.obs[qc_cols].head()


In [ ]:
summary_cols = [col for col in ["total_counts", "n_genes_by_counts", "pct_counts_mt"] if col in adata.obs.columns]
(
    adata.obs
    .groupby("sample_id", observed=False)[summary_cols]
    .median()
    .sort_index()
)

In [ ]:
sc.pp.filter_genes(adata, min_counts=3)
sc.pp.filter_cells(adata, min_counts=3)

adata.layers["counts"] = adata.X.copy()

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata.copy()

adata

In [ ]:
n_top_genes = min(2000, adata.n_vars)
sc.pp.highly_variable_genes(adata, n_top_genes=n_top_genes, flavor="seurat", subset=True)

sc.tl.pca(adata, svd_solver="arpack")
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=min(30, adata.obsm["X_pca"].shape[1]))
sc.tl.umap(adata, min_dist=0.3)
sc.tl.leiden(adata, resolution=1.0, key_added="leiden_1.0")

adata

In [ ]:
with plt.rc_context({"figure.figsize": (7, 5)}):
    sc.pl.umap(adata, color=["sample_id", "leiden_1.0"], wspace=0.4)

with plt.rc_context({"figure.figsize": (12, 6)}):
    sc.pl.embedding(adata, basis="spatial", color=["sample_id", "leiden_1.0"], s=4, frameon=False)


## Optional Save

Run the next cell if you want a processed `h5ad` written next to the extracted SpatialData cache.


In [ ]:
OUTPUT_H5AD = EXTRACT_ROOT / f"{Path(ZIP_PATH).stem}_{TABLE_NAME}.processed.h5ad"
adata.write_h5ad(OUTPUT_H5AD)
OUTPUT_H5AD

## Optional inspection

Use the next cell to inspect a specific table in detail. This is especially useful for the Visium HD archive, which contains multiple tables.

In [ ]:
table_name = TABLE_NAME
table = adata

print(table_name)
print(table)
table.obs.head()